In [5]:
import mysql.connector as db
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np

class db_communication:
    """
    ____________________________________________________________________________________________
    Benötigte Python-Module:
    - mysql
    - mysql-connector
    - pandas
    - numpy
    ____________________________________________________________________________________________
    Bietet eine API zur Kommnunikation mit der Fahrrad-Shop-Datenbank.
    Auf Basis des mysql-Moduls wird mit einer MariaDB kommuniziert.
    
    Durch print()-Ausgaben wird der Erfolg oder Misserfolg bei einem Funktionsaufruf ausgegeben.
    
    Die Klasse ist wie folgt zu nutzen:
    my_db = db_communication()
    -----
    my_db.user_func_xyz(args)
    -----
    my_db.close()
    """
    def __init__(self, config = None):
        """
        Default (config = None) wird die Verbindung zur Standard-Datenbank hergestellt.
        Alternativ kann ein config-dict übergeben werden
        config = {
            'user': '',
            'password': '',
            'host': '',
            'port': '',
            'database': ''
        }
        """
        self.__db = self.connect(config)
        if self.__db:
            self.__cursor = self.__db.cursor()
            print("Successfully connected.")
        else:
            print("... no connection ...")

    def connect(self, config=None) -> db.MySQLConnection:    
        """
        Stellt die Verbindung zur Datenbank her und nutzt die übergeben Parameter config.
        Return: 'db.MySQLConnection'.
        """
        if not config:
            config = {
            'user': 'root',
            'password': 'FDS-apm1',
            'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
            'port': '3306',
            'database': 'NeueDB'
            }

        try:
            return mysql.connector.connect(**config)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            print("Not connected")
            
    def close(self):
        """ 
        Schließt die Verbindung zur Datenbank.
        """
        if self.__db:
            self.__db.close()
            print("Connection closed")
        else:
            pass
    
    def read_tables(self) -> list:
        """
        Liest die Tabellen der Datenbank aus und gibt diese in Listen-Form zurück.
        """
        if self.__db:
            self.__cursor.execute("SHOW TABLES")

            tables = list()
            for x in self.__cursor:
                tables.append(x)

            return tables
        else:
            pass
        
    def insert_data(self, table_name, data_as_dict):
        """
        Der Tabelle 'table_name' wird eine neue Zeile hinzugefügt. 
        Der Inhalt der Zeile wird aus dem 'data_as_dict' erstellt. 
        Die Spalentennamen sind die 'keys' und die Zellenwerte die 'values' des Dictionaries.
        """
        try:
            sql = "INSERT INTO `" + table_name + "` ("
            values = ""
            for key in data_as_dict:
                sql += "`" + str(key) + "`, "
                if data_as_dict[key]: 
                    values += "'" + str(data_as_dict[key]) + "', "
                else:
                    values += "NULL , "
            sql = sql[:len(sql)-2]
            values = values[:len(values)-2]
            sql += ") VALUES (" + values + ");"
            
            print(sql)
            self.__cursor.execute(sql)
            self.__db.commit()
            return True
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
    
    def get_table(self, table_name, key = None, column = None) -> pd.DataFrame:
        """
        Die gewünschte Tabelle 'table_name' wird als pd.DataFrame zurückgegeben.
        """
        if key and column:
            pass # was war hier nochmal?
        elif column:
            self.__cursor.execute("SELECT {} FROM `{}`".format(column, table_name))
            rows = []
            for y in self.__cursor:
                rows.append(y)
            return pd.DataFrame(np.array(rows), columns = [column])
        elif key:
            pass # was war hier nochmal?
        else:
            self.__cursor.execute("SHOW COLUMNS FROM `{}`".format(table_name))
            columns = []
            for y in self.__cursor:
                columns.append(y[0])
            self.__cursor.execute("SELECT * FROM `{}`".format(table_name))
            rows = []
            for y in self.__cursor:
                rows.append(y)
            try: 
                df = pd.DataFrame(np.array(rows), columns=columns)
                return df
            except ValueError:
                return None
        
    def drop_table(self, name) -> bool:
        """
        Die Tabelle name wird gelöscht, allerdings nur unter der Bedingung, 
        dass die Tabelle keine Abhängigkeiten zu anderen Tabellen hat.
        """
        sql = "DROP TABLE %s" %str(name)
        try:
            # Execute the SQL command
            self.__cursor.execute(sql)
            # Commit your changes in the database
            self.__db.commit()
            print("Drop table: ", str(name))
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
        
        return True

    def show_columns(self, table_name):
        """
        Gibt die Spalten der gewünschten Tabelle über die print-Ausgabe aus.
        """
        tables = self.read_tables()
        if not any(name in s for s in tables):
            return self.__cursor.execute("SHOW COLUMNS FROM `{}`".format(x))
        else:
            print("Table", name, "already exists.")
            return False
        
    def storno(self, key):
        """
        Datenbank spezifische Funktion.
        Eine existierende Bestellung wird storniert, identifiziert an dem 'key' == 'Bestellung.id'. 
        Die Spalte "Storniert" wird auf True gesetzt.
        """
        
        sql = "UPDATE `Bestellungen` SET `Storniert`='1' WHERE `Bestellungen.id`= '" + key + "';"

        try:
            # Execute the SQL command
            self.__cursor.execute(sql)
            # Commit your changes in the database
            self.__db.commit()
            print("Update Bestellung:", str(key))
        except:
           # Rollback in case there is any error
            self.__db.rollback()
            print("Rollback, something went wrong.")
            return False
    
    def create_table(self, name, rows) -> bool:
        """
        Erstellt eine Tabelle mit dem Namen 'name' und den Spalten 'rows'.
        'rows'-Elemente müssen im MariaDB-Format die Einstellungen enthalten. 
        Bsp.1: 
        `Bauteil.id` INT NOT NULL AUTO_INCREMENT
        Bsp.2:
        FOREIGN KEY (`Baugruppe`) REFERENCES `Baugruppen`(`Baugruppe`)
        """
        tables = self.read_tables()
        if not any(name in s for s in tables):
            sql = "CREATE TABLE `" + name + "` ("
            for row in rows:     
                if rows.index(row) == (len(rows)-1):
                    sql += " " + row
                else:
                    sql += " " + row + ","
            sql += ")"
            print(sql)
            self.__cursor.execute(sql)
            print("Created table: ", name)
            return True
        else:
            print("Table", name, "already exists.")
            return False
        
    def reset_all_tables(self, are_u_sure = False):
        """
        Funktion um die Tabellen der Datenbank zu löschen und neu und leer wieder zu initialiseren.
        Die Variable 'are_u_sure' muss auf True gesetzt sein, um den Vorgang durchzuführen
        Der Aufbau der Tabellen ist fest niedergeschrieben und hat Abhängigkeiten wie FOREIGN KEY untereinander.
        Es ist auf die richtige Reihenfolge zu achten!
        Aus diesem Grund wird das Nutzen dieser Funktion empfohlen, um die Tabellen der Fahrrad-Shop-Datenbank zu löschen
        """
        config_kunde = [
            "`KundenNr` INTEGER NOT NULL AUTO_INCREMENT",
            "`Name` CHAR(30) NOT NULL",
            "`Vorname` CHAR(30) NOT NULL",
            "`Mail` CHAR(60)",
            "PRIMARY KEY (`KundenNr`)"
        ]

        config_auftrag = [ 
            "`AuftrNr` INTEGER NOT NULL AUTO_INCREMENT",
            "`Bestelldatum` DATE",
            "`Lieferdatum` DATE",
            "`KundenNr` INTEGER NOT NULL",
            "PRIMARY KEY (`AuftrNr`)",
            "FOREIGN KEY (`KundenNr`) REFERENCES `Kunde`(`KundenNr`)"
        ]

        config_merkmalcluster = [
            "`ClusterNr` INTEGER NOT NULL AUTO_INCREMENT",
            "`Merkmalsbezeichnung`CHAR(95)",
            "PRIMARY KEY (`ClusterNr`)"
        ]

        config_merkmale = [
            "`MerkmalNr` INTEGER NOT NULL AUTO_INCREMENT",
            "`Merkmalcluster` INTEGER NOT NULL",
            "`Bezeichnung` CHAR(95) NOT NULL",
            "`Preis` NUMERIC(7,2)",
            "PRIMARY KEY (`MerkmalNr`)",
            "FOREIGN KEY (`Merkmalcluster`) REFERENCES `Merkmalcluster`(`ClusterNr`)"
        ]
        
        config_konfiguration = [
            "`KonfigNr` INTEGER NOT NULL AUTO_INCREMENT",
            "`AuftragNr` INTEGER NOT NULL",
            "`MerkmalNr` INTEGER NOT NULL",
            "PRIMARY KEY (`KonfigNr`)",
            "FOREIGN KEY (`AuftragNr`) REFERENCES `Auftrag`(`AuftrNr`)",
            "FOREIGN KEY (`MerkmalNr`) REFERENCES `Merkmale`(`MerkmalNr`)"
        ]
        
        config_einzelteile = [ 
            "`EinzelteilNr` INTEGER NOT NULL AUTO_INCREMENT",
            "`Bezeichnung` CHAR(95) NOT NULL",
            "PRIMARY KEY (`EinzelteilNr`)"
        ]
        
        config_arbeitsschritt = [
            "`SchrittNr` INTEGER NOT NULL AUTO_INCREMENT",
            "`MerkmalNr` INTEGER NOT NULL",
            "`EinzelteilNr` INTEGER NOT NULL",
            "`Anzahl Einzelteil` INTEGER",
            "`Montagezeit` INTEGER",
            "`VorgaengerNr` INTEGER",
            "PRIMARY KEY (`SchrittNr`)",
            "FOREIGN KEY (`MerkmalNr`) REFERENCES `Merkmale`(`MerkmalNr`)",
            "FOREIGN KEY (`EinzelteilNr`) REFERENCES `Einzelteile`(`EinzelteilNr`)"
        ]
        
        config_reihenfolgevorgabe = [
            "`SchrittNr` INTEGER NOT NULL",
            "`NachfolgerNr` INTEGER NOT NULL",
            "PRIMARY KEY (`SchrittNr`)",
            "FOREIGN KEY (`SchrittNr`) REFERENCES `Arbeitsschritt`(`SchrittNr`)",
            "FOREIGN KEY (`NachfolgerNr`) REFERENCES `Arbeitsschritt`(`SchrittNr`)"
        ]


        if are_u_sure:
            list_names_tables = ["Kunde","Auftrag","Merkmalcluster","Merkmale","Konfiguration","Einzelteile","Arbeitsschritt","Reihenfolgevorgabe"]
            list_config_tables=[config_kunde,config_auftrag,config_merkmalcluster,config_merkmale,config_konfiguration,config_einzelteile,config_arbeitsschritt,config_reihenfolgevorgabe]
            table_names = list_names_tables[::-1]
            print(table_names)
            for t in table_names:
                self.drop_table(t)

        for idx in range(len(list_config_tables)):
            self.create_table(list_names_tables[idx],list_config_tables[idx])

In [6]:
#Reset the DB to an empty DB
my_db = db_communication()
my_db.reset_all_tables(True)
my_db.close()

Successfully connected.
['Reihenfolgevorgabe', 'Arbeitsschritt', 'Einzelteile', 'Konfiguration', 'Merkmale', 'Merkmalcluster', 'Auftrag', 'Kunde']
Drop table:  Reihenfolgevorgabe
Drop table:  Arbeitsschritt
Drop table:  Einzelteile
Drop table:  Konfiguration
Drop table:  Merkmale
Drop table:  Merkmalcluster
Drop table:  Auftrag
Drop table:  Kunde
CREATE TABLE `Kunde` ( `KundenNr` INTEGER NOT NULL AUTO_INCREMENT, `Name` CHAR(30) NOT NULL, `Vorname` CHAR(30) NOT NULL, `Mail` CHAR(60), PRIMARY KEY (`KundenNr`))
Created table:  Kunde
CREATE TABLE `Auftrag` ( `AuftrNr` INTEGER NOT NULL AUTO_INCREMENT, `Bestelldatum` DATE, `Lieferdatum` DATE, `KundenNr` INTEGER NOT NULL, PRIMARY KEY (`AuftrNr`), FOREIGN KEY (`KundenNr`) REFERENCES `Kunde`(`KundenNr`))
Created table:  Auftrag
CREATE TABLE `Merkmalcluster` ( `ClusterNr` INTEGER NOT NULL AUTO_INCREMENT, `Merkmalsbezeichnung`CHAR(95), PRIMARY KEY (`ClusterNr`))
Created table:  Merkmalcluster
CREATE TABLE `Merkmale` ( `MerkmalNr` INTEGER NOT NULL

In [7]:
#Fill DB with Default Values
import os
from path import Path
path = os.getcwd()
path_merkmalcluster = Path(path) / "Bike_DB/merkmalcluster.csv"
path_merkmale = Path(path) / "Bike_DB/merkmale.csv"
path_einzelteile = Path(path) / "Bike_DB/einzelteile.csv"

df_merkmalcluster = pd.read_csv(path_merkmalcluster, sep=',')
df_merkmale = pd.read_csv(path_merkmale, sep=',')
df_einzelteile = pd.read_csv(path_einzelteile, sep=',')

df_merkmalcluster.dropna(axis = 1, inplace=True, how = "all")
df_merkmale.dropna(axis = 1, inplace=True, how = "all")
df_einzelteile.dropna(axis = 1, inplace=True, how = "all")

columns_merkmalcluster = [df_merkmalcluster.columns][0]
columns_merkmale = [df_merkmale.columns][0]
columns_einzelteile = [df_einzelteile.columns][0]

column_merkmalcluster = ()
column_merkmale = ()
column_einzelteile = ()
for i in range(len(columns_merkmalcluster)):
     column_merkmalcluster += (columns_merkmalcluster[i],)

for i in range(len(columns_merkmale)):
     column_merkmale += (columns_merkmale[i],)

for i in range(len(columns_einzelteile)):
     column_einzelteile += (columns_einzelteile[i],)

my_db = db_communication()

data_merkmalcluster = df_merkmalcluster.values.tolist()
data_merkmale = df_merkmale.values.tolist()
data_einzelteile = df_einzelteile.values.tolist()

for i in range(0,len(data_merkmalcluster)):
    data_merkmalcluster_as_dict = {}
    for j in range(len(column_merkmalcluster)):
            data_merkmalcluster_as_dict[column_merkmalcluster[j]] = data_merkmalcluster[i][j]
    my_db.insert_data("Merkmalcluster", data_merkmalcluster_as_dict)

for i in range(0,len(data_merkmale)):
    data_merkmale_as_dict = {}
    for j in range(len(column_merkmale)):
            data_merkmale_as_dict[column_merkmale[j]] = data_merkmale[i][j]
    my_db.insert_data("Merkmale", data_merkmale_as_dict)

for i in range(0,len(data_einzelteile)):
    data_einzelteile_as_dict = {}
    for j in range(len(column_einzelteile)):
            data_einzelteile_as_dict[column_einzelteile[j]] = data_einzelteile[i][j]
    my_db.insert_data("Einzelteile", data_einzelteile_as_dict)

my_db.close()

Successfully connected.
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('1', 'Rahmen');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('2', 'Federung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('3', 'Lackierung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('4', 'Lenker');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('5', 'Felgen');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('6', 'Radbereifung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('7', 'Gangschaltung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('8', 'Sattel');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('9', 'Bremsen');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('10', 'Beleuchtung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkma

INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('800003', '8', 'Trekkingsattel Herren', '26.95');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('800004', '8', 'Trekkingsattel Damen', '29.99');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('800005', '8', 'Rennradsattel Herren', '129.95');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('800006', '8', 'Rennradsattel Damen', '164.99');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('800007', '8', 'Sattel Junior', '9.9');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('900001', '9', 'Felgenbremse Standard', '26.9');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('900002', '9', 'Felgenbremse Trekking', '99.99');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`,

INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7009', 'Shimano Dura Ace Di2 RD-R9150 Shadow');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7010', 'Shimano CN-HG901');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7011', 'Shimano CS-R9100');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7012', 'Shimano BB-R9100');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7013', 'XX1 Eagle DUB');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7014', 'XX1 Eagle Triggershifter');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7015', 'X-Horizon, X-Actuation, X-Sync, Roller Bearing Clutch, Cage Lock');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7016', 'XX1 Eagle Kette');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('7017', 'XG-1295 Eagle Kassette');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('

In [4]:
import random
def randomElementInList(values):
    """
    Auswahl eines zufälligen Elements aus dem Wertebereich values
    Bsp.: ['Basic','Holland','BMX','Klassisch','Einrad'] ->  return = 'BMX'
    """
    return random.choice(values)

In [5]:
randomElementInList(['Basic','Holland','BMX','Klassisch','Einrad'])

'Klassisch'

In [6]:
def randomNumber(start,stop=None,step=1):
    """
    Generierung einer zufälligen Zahl nach random.randrange():
    Choose a random item from range(start, stop[, step])
    """
    if start != stop:
        return random.randrange(start=start,stop=stop,step=step)
    else:
        return start

In [7]:
randomNumber(1,20,1)

9

In [8]:
import time 
import datetime
def randomDate(start, end=datetime.datetime.now().strftime("%d.%m.%Y"), format="%d.%m.%Y", prop=random.random()):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

In [9]:
randomDate("12.10.2020","12.10.2022","%d.%m.%Y",random.random())

'06.11.2021'

In [10]:
def randomName():
    preNames = ["Tana","Francoise","Emmie","Stuart","Cedrick","Tiffani","Kym","Woodrow","Jolene","Autumn","Moriah","Claud","Antoinette",
                "Gisele","Chrissy","Ellena","Hilton","Susanne","Lindy","Nakesha","Rebecca","Lavenia","Donya","Claudia","Dreama","Alla",
                "Freeman","Weston","Jacki","Helene"]
    lastName = ["Allinder","Longley","Dobyns","Coldiron","Justin","Munroe","Boll","Clingerman","Pries","Santangelo","Miller","Bohman","Meachum",
                "Kamer","Waits","Polin","Suitt","Saulter","Pelt","Dapolito","Guild","Beeks","Hollingshead","Schissler","Cardamone","Sypher",
                "Fannin","Belizaire","Burleigh","Eslinger"]
    return randomElementInList(preNames), randomElementInList(lastName)

In [11]:
randomName()

('Alla', 'Longley')

In [12]:
def randomMail(preName,lastName):
    providers = ['gmail.com','web.de','telekom.de','apple.com','fh-bielefeld.de']
    mail = "{0}.{1}@{2}".format(preName.lower(),lastName.lower(),randomElementInList(providers).lower())
    return mail

In [13]:
randomMail(*randomName())

'claud.longley@gmail.com'

In [14]:
def randomBike(featureTableName='Merkmale',featureClusterTableName='Merkmalcluster'):
    """
    Generierung eines zufälligen Fahrrad auf Basis der Inhalte aus den Tabellen:
    - featureCluster (Bestandteile eines Fahrrads: Rahmen, Gabel, etc.)
    - feature        (Wie diese Bestandteile umgesetzt werden: Trofeo 5, Carbon Öldruckstoßdämpfer, etc.)
    Und Bestimmung des Gesamtpreises anhand gewählter Merkmale
    """
    # get information about the parts to build a bike from DB
    df_features = my_db.get_table(featureTableName)
    df_featureCluster = my_db.get_table(featureClusterTableName)
    df_featureCluster.set_index('ClusterNr',inplace=True)
    
    # merge both dfs
    arrays = [df_features['Merkmalcluster'].replace(df_featureCluster.to_dict()['Merkmalsbezeichnung']).values, 
              df_features['MerkmalNr'].values]
    tuples = list(zip(*arrays))
    index = pd.MultiIndex.from_tuples(tuples, names=['Merkmalcluster', 'MerkmalNr'])
    df = df_features.set_index(index)
    
    # build bike
    bike = []
    bikePrice = 0
    for cluster in df['Merkmalcluster'].unique():
        # get set of Merkmale
        setMerkmale = df.query("Merkmalcluster == " + str(cluster))['MerkmalNr'].values    
        # ignoe 'no ...' part (marked with MerkmalNr : 'xxx99999')
        unvalidClusters = [element for element in setMerkmale.astype(str) if '99999' in element]
        setMerkmale = setMerkmale[setMerkmale.astype(str) != unvalidClusters]
        # select random bike part
        bikePart = randomElementInList(setMerkmale)
        # get price of this part
        pricePart = df.swaplevel().loc[int(bikePart)]['Preis'].astype(float).iloc[0]
        # sum it up to get the total bike price
        bikePrice += pricePart
        bike.append(bikePart)
    return bike, bikePrice

In [15]:
randomBike()

OperationalError: 2055: Lost connection to MySQL server at 'min-ifm-xdm.ad.fh-bielefeld.de:3306', system error: 10038 Ein Vorgang bezog sich auf ein Objekt, das kein Socket ist

In [ ]:
def fillCustomerTable(n,columnNamesDB=['Name','Vorname','Mail'],includeID=True):
    """
    Hinzufügen von zufällig generierten Einträgen in der DB-Tabelle 'Kunde'
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    columnNamesDB - Spaltennamen der Kunden-Tabelle in der DB (mit Ausnahme der KundenNr; Reihenfolge beachten!) 
    includeID - Ob KundenNr seperat mit angelegt werden soll
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    # Kundentabelle füllen
    table_name = 'Kunde'
    customerId = 'KundenNr'
    if includeID:
        customer_keys = np.concatenate(([customerId],columnNamesDB)) 
    else:
        customer_keys = columnNamesDB
    for i in range(n):
        customer_name = randomName()
        customer_values = i+1, customer_name[0], customer_name[1], randomMail(*customer_name)
        if includeID:
            customer_dict = dict(zip(customer_keys,customer_values))
        else:
            customer_dict = dict(zip(customer_keys,customer_values[1:]))
        my_db.insert_data(table_name,customer_dict)

In [ ]:
fillCustomerTable(1,includeID=False)

In [ ]:
def generateConfigurationData(n,columnNamesDB=['AuftragNr','MerkmalNr'],
                              tableName_Order='Auftrag',tableName_Config='Konfiguration',
                              configId_name = 'KonfigNr',includeID=True):
    """
    Aufgrund der Abhängigkeiten der Tabellen 'Auftrag' und 'Konfiguration' mit den Spalten 'Gesamtpreis' <-> 'AuftragNr', 
    wird in dieser Funktion der Output für die Konfiguration inkl. Informationen für den Auftragfür ein späteres Einfügen 
    in die DB erzeugt.
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    columnNamesDB - Spaltennamen der Kunden-Tabelle in der DB (mit Ausnahme der KundenNr; Reihenfolge beachten!) 
    tableName_Order - Name der Tabelle für die Aufträge
    tableName_Config - Name der Tabelle für die Konfiguration
    configId_name - Name des PrimaryKeys der Konfiguration-Tabelle
    includeID - Ob KonfigNr seperat mit angelegt werden soll
    """
    # create keys
    if includeID:
        config_keys = np.concatenate(([configId_name],columnNamesDB)) 
    else:
        config_keys = columnNamesDB

    # get old entries in order to appending new ones
    df_before = my_db.get_table(tableName_Order)
    if df_before is None:
        start = 1
    else:
        start = df_before.iloc[:,0].max() + 1 #max orderId + 1
    n += start

    order = []
    config_dict = []
    # iterate through all orders that should be created
    for orderId in range(start,n):
        # build random bike
        bike,bikePrice = randomBike()
        # keep orderId and bikePrice for later order fillings
        order.append(dict(zip(('orderId','bikePrice'),(orderId,bikePrice))))
        # iterate through each bike part
        for partId, featureId in enumerate(bike):
            # generate values
            config_values = orderId*100+partId, orderId, featureId
            # keep the config_dict for later inserts 
            if includeID:
                config_dict.append(dict(zip(config_keys,config_values)))
            else:
                config_dict.append(dict(zip(config_keys,config_values[1:])))
    # to data frame          
    df_order = pd.DataFrame(order)
    return config_dict, df_order

In [ ]:
def generateOrderData(df_order,columnNamesDB=['Bestelldatum','Lieferdatum','KundenNr','Gesamtpreis'],
                      table_name='Auftrag',orderId_name='AuftrNr', includeID=True, 
                      customerTableName='Kunde',customerId='KundenNr',startDate='2020-01-01'):
    """
    Erstellen von zufälligen Auftragsinhalten in Abhängigkeit zuvor erstellter Konfigurationen
    
    df_order - DataFrames welches von dem Konfigurator zur Anlegung eines Auftrags und zur Bestimmung des Gesamtpreises verwendet wird
    columnNamesDB - Spaltennamen der Auftrag-Tabelle in der DB (mit Ausnahme der AuftrNr; Reihenfolge beachten!) 
    table_name - Name der Tabelle für die Aufträge
    configId_name - Name des PrimaryKeys der Auftrag-Tabelle
    includeID - Ob AuftrNr seperat mit angelegt werden soll
    customerTableName - Name der DB-Tabelle 'Kunde'
    customerId - Name der Identifikationsnummer in customerTable
    startDate - Startdatum zur zufälligen Erzeugung
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    # Read relevant range of values for existing customers
    df_customers = my_db.get_table(customerTableName)
    
    # create keys
    if includeID:
        order_keys = np.concatenate(([orderId_name],columnNamesDB)) 
    else:
        order_keys = columnNamesDB
        
    # create values and fill order_dict_lst
    order_dict = []
    for _,order in df_order.iterrows():
        orderId = order[0].astype(int)
        orderPrice = order[1].astype(float)
        
        rand_orderDate = randomDate(start=startDate,
                                    end=datetime.datetime.now().strftime("%Y-%m-%d"),
                                    format="%Y-%m-%d",
                                    prop=random.random())
        rand_delivDate = datetime.datetime.strftime(datetime.datetime.strptime(rand_orderDate,"%Y-%m-%d") 
                                                    + datetime.timedelta(days=14),"%Y-%m-%d")
        rand_customerId = randomNumber(start=df_customers[customerId].astype(int).min(),
                                        stop=df_customers[customerId].astype(int).max())
        order_values = orderId, rand_orderDate, rand_delivDate, rand_customerId, orderPrice
        if includeID:
            order_dict.append(dict(zip(order_keys,order_values)))
        else:
            order_dict.append(dict(zip(order_keys,order_values[1:])))
    return order_dict

In [ ]:
def fillConfigAndOrderTable(n,tableName_Order='Auftrag',tableName_Config='Konfiguration'):
    """
    Tatsächliches Befüllen der beiden Tabellen 'Auftrag' und 'Konfiguration'
    Entspricht der Simulation eines (oder mehreren) Kunden, die sich ein Fahrrad in einem Online-Konfigurator zusammenstellen 
    lassen und bestellen, wodurch ein Auftrag angelegt wird 
    
    n - Anzahl der zu erstellenden Aufträge inkl. Konfigurationen
    """
    config_dict, df_order = generateConfigurationData(n)
    order_dict = generateOrderData(df_order)
    for order_dict_part in order_dict:
        my_db.insert_data(tableName_Order,order_dict_part)
    for config_dict_part in config_dict:
        my_db.insert_data(tableName_Config,config_dict_part)

In [ ]:
fillConfigAndOrderTable(2)